In [ ]:
%%time
"""
    Atividade Final.

    Autor:
        Richard de Andrade
        Flávio do Nascimento Bilancieri
    E-mail: 
        Richard - 1407221@sga.pucminas.br
        Flávio - 1407220@sga.pucminas.br
    Data:
        2022-11-14
"""


# Conexão spark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

spark = SparkSession\
    .builder\
    .appName("Prep-Orq-Exerc-Final")\
    .getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", 1)

In [ ]:
%%time
# Trata as colunas do arquivo titanic.csv
schema_arq = (
    StructType([ 
        StructField("PassengerId", IntegerType(), True),
        StructField("Survived", IntegerType(), True),
        StructField("Pclass", IntegerType(), True),
        StructField("Name",StringType(),True), 
        StructField("Sex",StringType(),True), 
        StructField("Age", FloatType(), True),
        StructField("SibSp", IntegerType(), True),
        StructField("Parch", IntegerType(), True),
        StructField("Ticket",StringType(),True), 
        StructField("Fare", FloatType(), True),
        StructField("Cabin",StringType(),True), 
        StructField("Embarked",StringType(),True)
    ])
)

In [ ]:
%%time
params = {"header": True, "schema": schema_arq, "sep": ";",  "encoding": "latin1"}

# Le o arquivo
df = spark.read.csv("s3://airflow-logs-452490318365/arquivo/", **params)

# Cria uma temp view
df.createOrReplaceTempView("titanic")

# Faz uma contagem dos sobreviventes
survived = (
    spark.sql('''
    select survived, count(1) as qtd
    from   titanic
    group  by survived
    ''')
)

# Armazena o resultado em um arquivo parquet
survived.write.parquet("s3://airflow-logs-452490318365/arquivo/")

In [ ]:
%%time
spark.sql('drop table if exists tb_tinatic_survived')

# Cria uma tabela apontando para o arquivo parquet
spark.sql('''
create external table tb_tinatic_survived 
(
  survived int,
  qtd      int
)
stored as parquet
tblproperties ('parquet.compression' = 'snappy')
location "s3://airflow-logs-452490318365/arquivo/"
''')
  
# Describe na tabela para mostrar que não está vazia
spark.sql('describe formatted tb_tinatic_survived').show(n=200, truncate=False)

In [ ]:
# Mostra o resultado da tabela
spark.table('tb_tinatic_survived').show(n=5, truncate=False)